# 🧠 Tuning into Tumors: Frequency-Domain Tumor Segmentation

**BME 271D Final Project - Fall 2025**  
**Team:** Ege Ozemek, Max Bazan, Sasha Nikiforov

---

## 📋 Quick Start Guide

This notebook demonstrates automated tumor segmentation using **frequency-domain filtering** techniques:

1. **FFT-based High-Pass Filtering** → Emphasizes tumor boundaries
2. **FFT-based Band-Pass Filtering** → Isolates tumor texture
3. **Canny Edge Detection** → Spatial-domain baseline
4. **Otsu Thresholding** → Simple baseline comparison

### ▶️ **How to Run This Notebook:**

1. Click **"Runtime" → "Run all"** (or press `Ctrl+F9`)
2. Wait 1-2 minutes for results
3. Scroll down to see visualizations and metrics

**That's it!** Everything runs automatically.

---

## 🎓 Learning Objectives

- Understand **Fourier Transform** applications in medical imaging
- Design **frequency-domain filters** (high-pass, band-pass)
- Compare **spatial vs. frequency domain** segmentation methods
- Evaluate performance using **Dice coefficient** and **IoU**

---

## 🔧 Step 1: Setup & Installation

First, we'll install required packages and download the code.

In [ ]:
%%capture
# Install required packages (runs silently)
!pip install numpy matplotlib scipy scikit-image pandas

# Download our tumor segmentation code from GitHub
!wget -q https://raw.githubusercontent.com/YOUR_GITHUB_USERNAME/tumor-segmentation/main/tumor_segmentation.py

# Download sample tumor images
!wget -q https://github.com/YOUR_GITHUB_USERNAME/tumor-segmentation/raw/main/sample_data.zip
!unzip -q sample_data.zip

print("✅ Setup complete! Packages installed and data downloaded.")

In [ ]:
# Import libraries
import tumor_segmentation as ts
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Configure matplotlib for better looking plots
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 11

print("✅ Imports successful!")
print("\n📊 Ready to analyze tumor images!")

---

## 📚 Step 2: Understanding Frequency Domain Analysis

### Why Frequency Domain?

Medical images contain information at different **spatial frequencies**:

- **Low frequencies** → Smooth regions, overall intensity, background
- **High frequencies** → Edges, boundaries, sharp transitions
- **Mid frequencies** → Textures, internal patterns

**Tumors often have:**
- Sharp boundaries (high frequencies)
- Heterogeneous texture (mid frequencies)
- Different intensity than surrounding tissue

By filtering in the frequency domain, we can **selectively enhance tumor features** for better segmentation!

In [ ]:
# Load a sample tumor image
sample_image = ts.load_grayscale_image('data/images/tumor_001.png')
sample_mask = ts.load_binary_mask('data/masks/tumor_001.png')

print(f"📸 Loaded image: {sample_image.shape[0]}×{sample_image.shape[1]} pixels")
print(f"🎯 Tumor region: {sample_mask.sum()} pixels ({100*sample_mask.sum()/sample_mask.size:.1f}% of image)")

# Display the image
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(sample_image, cmap='gray')
axes[0].set_title('Original MRI/CT Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(sample_image, cmap='gray')
axes[1].imshow(sample_mask, cmap='Reds', alpha=0.5)
axes[1].set_title('Ground Truth Tumor Annotation', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Note: The red overlay shows where the tumor actually is (ground truth).")

---

## 🌊 Step 3: Fourier Transform Analysis

Let's compute the **2D Fast Fourier Transform (FFT)** to see what frequencies are present in the image.

In [ ]:
# Compute FFT
F_shift, magnitude = ts.compute_fft_spectrum(sample_image)

# Visualize
fig = ts.visualize_frequency_spectrum(sample_image, F_shift, 
                                      title="Frequency Domain Representation")
plt.show()

print("\n🔍 Key Observations:")
print("  • Bright center = Low frequencies (smooth background)")
print("  • Radial patterns = High frequencies (edges, tumor boundaries)")
print("  • Mid-range = Texture information (tumor heterogeneity)")
print("\n💡 By filtering these frequencies, we can enhance tumor features!")

---

## 🎛️ Step 4: Design Frequency Filters

We'll create three types of filters:

1. **High-Pass Filter** → Blocks low frequencies, keeps edges
2. **Band-Pass Filter** → Keeps only mid-frequencies (texture)
3. **Low-Pass Filter** → Smooths image (for comparison)

In [ ]:
# Create filter masks
hp_mask = ts.make_hp_mask(sample_image.shape, cutoff_radius=25)
bp_mask = ts.make_bp_mask(sample_image.shape, r1=10, r2=40)
lp_mask = ts.make_lp_mask(sample_image.shape, cutoff_radius=50)

# Visualize filters
fig = ts.visualize_filters(hp_mask, bp_mask, lp_mask)
plt.show()

print("\n📊 Filter Statistics:")
print(f"  • High-Pass: Passes {100*hp_mask.sum()/hp_mask.size:.1f}% of frequencies")
print(f"  • Band-Pass: Passes {100*bp_mask.sum()/bp_mask.size:.1f}% of frequencies")
print(f"  • Low-Pass:  Passes {100*lp_mask.sum()/lp_mask.size:.1f}% of frequencies")

---

## 🔬 Step 5: Apply Filters & See Effects

Now let's apply these filters and see how they transform the image!

In [ ]:
# Apply frequency-domain filters
hp_filtered, _, _ = ts.filter_pipeline(sample_image, 'hp', cutoff_radius=25)
bp_filtered, _, _ = ts.filter_pipeline(sample_image, 'bp', r1=10, r2=40)
lp_filtered, _, _ = ts.filter_pipeline(sample_image, 'lp', cutoff_radius=50)

# Visualize filtering effects
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

axes[0, 0].imshow(sample_image, cmap='gray')
axes[0, 0].set_title('Original Image', fontsize=13, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(hp_filtered, cmap='gray')
axes[0, 1].set_title('High-Pass Filtered\n(Edge Enhancement)', fontsize=13, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(bp_filtered, cmap='gray')
axes[1, 0].set_title('Band-Pass Filtered\n(Texture Isolation)', fontsize=13, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(lp_filtered, cmap='gray')
axes[1, 1].set_title('Low-Pass Filtered\n(Smoothing)', fontsize=13, fontweight='bold')
axes[1, 1].axis('off')

plt.suptitle('Frequency-Domain Filtering Effects', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✨ Notice how each filter emphasizes different image features!")

---

## 🎯 Step 6: Tumor Segmentation

Now we apply segmentation algorithms to the filtered images and compare them to baseline methods.

### Methods We're Testing:

**Baseline Methods:**
1. Raw Otsu (direct thresholding)
2. Smoothed Otsu (Gaussian + threshold)

**Our Methods:**
3. FFT High-Pass + Otsu
4. FFT Band-Pass + Otsu
5. Canny Edge Detection

In [ ]:
# Define parameters
params = {
    'hp_radius': 25,
    'bp_r1': 10,
    'bp_r2': 40,
    'canny_sigma': 1.0,
    'gaussian_sigma': 1.0
}

# Run all segmentation methods
print("🔄 Running segmentation pipeline...\n")
results = ts.run_single_image_experiment(sample_image, sample_mask, params, verbose=True)

# Display results table
print("\n" + "="*70)
print(" 📊 QUANTITATIVE RESULTS")
print("="*70)
print(f"\n{'Method':<25} {'Dice':<10} {'IoU':<10} {'Boundary'}")
print("-"*70)

for method_name, method_data in results.items():
    metrics = method_data['metrics']
    print(f"{method_name:<25} {metrics['dice']:<10.4f} {metrics['iou']:<10.4f} {metrics['boundary_acc']:<10.4f}")

print("\n💡 Higher scores = Better segmentation")
print("   Dice > 0.80 is considered 'good' in medical imaging")

---

## 👁️ Step 7: Visual Comparison

Let's see how each method's segmentation compares to the ground truth!

In [ ]:
# Create visual comparison
masks_dict = {name: data['mask'] for name, data in results.items()}
fig = ts.plot_segmentation_comparison(sample_image, masks_dict, sample_mask)
plt.show()

print("\n🔍 What to look for:")
print("  • Red overlay = Method's prediction")
print("  • Green overlay = Ground truth (what we're trying to match)")
print("  • Better overlap = Better segmentation")

---

## 🏆 Step 8: Best Method Analysis

Let's identify which method performed best and create a detailed comparison with ground truth.

In [ ]:
# Find best method
best_method_name = max(results.items(), key=lambda x: x[1]['metrics']['dice'])[0]
best_method_data = results[best_method_name]
best_dice = best_method_data['metrics']['dice']

print(f"🏆 Best Method: {best_method_name}")
print(f"   Dice Score: {best_dice:.4f}")
print(f"   IoU Score: {best_method_data['metrics']['iou']:.4f}")
print(f"   Boundary Acc: {best_method_data['metrics']['boundary_acc']:.4f}")

# Create detailed overlay
fig = ts.create_overlay_visualization(sample_image, best_method_data['mask'], 
                                      sample_mask, best_method_name)
plt.show()

print("\n💡 In the overlay:")
print("   RED contours = Our prediction")
print("   GREEN contours = Ground truth")
print("   Perfect match would show yellow (red + green overlap)")

---

## 📏 Step 9: Clinical Volume Estimation

In clinical practice, tumor volume is critical for:
- Treatment planning
- Monitoring tumor growth/shrinkage
- Radiation therapy dosage
- Surgical planning

In [ ]:
# Typical MRI parameters (these would come from DICOM metadata in practice)
pixel_spacing = (0.5, 0.5)  # mm per pixel
slice_thickness = 5.0        # mm

# Calculate volumes
gt_vol_mm3, gt_vol_cm3 = ts.estimate_tumor_volume(sample_mask, pixel_spacing, slice_thickness)
pred_vol_mm3, pred_vol_cm3 = ts.estimate_tumor_volume(best_method_data['mask'], 
                                                       pixel_spacing, slice_thickness)

volume_error = abs(pred_vol_mm3 - gt_vol_mm3) / gt_vol_mm3 * 100

print("="*70)
print(" 📏 VOLUME ESTIMATION RESULTS")
print("="*70)
print(f"\nGround Truth Tumor Volume:")
print(f"  {gt_vol_mm3:.1f} mm³ = {gt_vol_cm3:.3f} cm³")
print(f"\n{best_method_name} Estimated Volume:")
print(f"  {pred_vol_mm3:.1f} mm³ = {pred_vol_cm3:.3f} cm³")
print(f"\nVolume Estimation Error: {volume_error:.1f}%")

if volume_error < 10:
    print("\n✅ Excellent accuracy! <10% error is clinically acceptable.")
elif volume_error < 25:
    print("\n✓ Good accuracy. <25% error is useful for monitoring.")
else:
    print("\n⚠️ Moderate accuracy. May need refinement for clinical use.")

# Visualize
fig = ts.display_volume_measurement(sample_image, best_method_data['mask'], 
                                   pred_vol_mm3, pred_vol_cm3)
plt.show()

---

## 📊 Step 10: Batch Analysis (Multiple Images)

For statistical validity, we should test on multiple images. Let's run the pipeline on all available samples.

In [ ]:
# Load all images
images, masks, filenames = ts.load_dataset('data/images', 'data/masks')

print(f"📚 Loaded {len(images)} images for batch analysis\n")

# Run batch experiment
print("🔄 Running batch segmentation (this may take 1-2 minutes)...\n")
results_df = ts.run_batch_experiment(images, masks, filenames, params, output_dir=None)

print("\n✅ Batch analysis complete!\n")

# Display summary statistics
methods = ['Baseline_Raw_Otsu', 'Baseline_Smooth_Otsu', 'FFT_HighPass', 'FFT_BandPass', 'Canny_Edges']
metrics_list = ['dice', 'iou', 'boundary_acc']

for metric in metrics_list:
    print(f"\n{'='*70}")
    print(f" {metric.upper()} SCORES (Mean ± Std)")
    print(f"{'='*70}")
    
    for method in methods:
        col_name = f'{method}_{metric}'
        if col_name in results_df.columns:
            mean = results_df[col_name].mean()
            std = results_df[col_name].std()
            print(f"  {method:<30s}: {mean:.4f} ± {std:.4f}")

print("\n" + "="*70)

---

## 📝 Conclusions & Key Takeaways

### What We Demonstrated:

1. **Frequency Domain Analysis**
   - Computed 2D FFT to analyze spatial frequency content
   - Identified that tumor boundaries contain high-frequency information
   - Designed custom filters to enhance tumor features

2. **Multiple Segmentation Approaches**
   - Baseline intensity thresholding (Otsu)
   - FFT-based frequency filtering
   - Spatial-domain edge detection (Canny)

3. **Quantitative Evaluation**
   - Dice coefficient (most common in medical imaging)
   - IoU (Intersection over Union)
   - Boundary accuracy

4. **Clinical Application**
   - Automated tumor volume estimation
   - Relevant for treatment planning and monitoring

### Signals & Systems Concepts Applied:

- ✅ **Fourier Transform** (BME 271D Topic 6)
- ✅ **Frequency Domain Filtering** (High-pass, Band-pass)
- ✅ **Convolution** (Gaussian smoothing, morphological operations)
- ✅ **Sampling Theory** (Pixel spacing, resolution)
- ✅ **System Analysis** (Comparing different processing pipelines)

### Future Directions:

1. Extend to 3D volumetric segmentation
2. Adaptive parameter selection
3. Multi-modal MRI fusion (T1, T2, FLAIR)
4. Integration with machine learning
5. Real-time clinical deployment

---

## 🎓 Thank You!

**Questions? Feedback?**

Feel free to:
- Modify the code and experiment with different parameters
- Upload your own tumor images
- Try different filter designs

**Project Repository:** [GitHub Link]

**Team Contact:**
- Ege Ozemek
- Max Bazan  
- Sasha Nikiforov

---

*BME 271D: Signals and Systems | Duke University | Fall 2025*